Load Thermal-FIST library and the common headers

In [1]:
#include "inc/loadFISTlibs.h"

###############################################################################
#                                                                             #
# This is Thermal-FIST version 1.2.2                                          #
#                                                                             #
# Copyright (c) 2019 Volodymyr Vovchenko <vovchenko@fias.uni-frankfurt.de>    #
#                                                                             #
# Distributed under the GNU General Public License 3.0 (GPLv3 or later)       #
#                                                                             #
# Please cite when using this code:                                           #
# V. Vovchenko, H. Stoecker, arXiv:1901.05249 [nucl-th]                       #
#                                                                             #
# The latest version is available at https://github.com/vlvovch/Thermal-FIST  #
#                                       

In [2]:
using namespace std;

In [3]:
using namespace thermalfist;

In [4]:
bool useWidth  = true;
bool useQStats = true;

double Tch = 0.155;

In [5]:
// Particle list (no nuclei)
//ThermalParticleSystem parts(string(INPUT_FOLDER) + "/list/PDG2014/list-withnuclei.dat");
ThermalParticleSystem parts(string(INPUT_FOLDER) + "/list/PDG2014/list.dat");

Fill the list of the considered hadron species

In [6]:
vector<int> pdgsCE;
vector<string> namesCE;

pdgsCE.push_back(111);
namesCE.push_back("pi0");

pdgsCE.push_back(211);
namesCE.push_back("pi+");

pdgsCE.push_back(321);
namesCE.push_back("K+");

pdgsCE.push_back(310);
namesCE.push_back("K0S");

pdgsCE.push_back(221);
namesCE.push_back("eta");

pdgsCE.push_back(323);
namesCE.push_back("KStar+");

pdgsCE.push_back(333);
namesCE.push_back("phi");

pdgsCE.push_back(2212);
namesCE.push_back("p");

pdgsCE.push_back(2112);
namesCE.push_back("n");

pdgsCE.push_back(3122);
namesCE.push_back("Lambda");

pdgsCE.push_back(3222);
namesCE.push_back("Sigma+");

pdgsCE.push_back(3112);
namesCE.push_back("Sigma+");

pdgsCE.push_back(3212);
namesCE.push_back("Sigma0");

pdgsCE.push_back(3322);
namesCE.push_back("Ksi0");

pdgsCE.push_back(3312);
namesCE.push_back("Ksi-");

pdgsCE.push_back(3334);
namesCE.push_back("Omega");

/*pdgsCE.push_back(1000010020);
namesCE.push_back("d");

pdgsCE.push_back(1000020030);
namesCE.push_back("He3");

pdgsCE.push_back(1010010030);
namesCE.push_back("H3La");

pdgsCE.push_back(1000020040);
namesCE.push_back("He4");*/

Compute the limiting GCE yield values

In [7]:
vector<double> yieldsGCE;
{
    ThermalModelIdeal modelgce(&parts);

    if (useWidth)
        modelgce.SetUseWidth(ThermalParticle::eBW);
    else
        modelgce.SetUseWidth(ThermalParticle::ZeroWidth);
    modelgce.SetStatistics(useQStats);
    // Include quantum statistics for mesons only
    if (useQStats){
        for (int i = 0; i < modelgce.TPS()->Particles().size(); ++i) {
            ThermalParticle &part = modelgce.TPS()->Particle(i);
            if (part.BaryonCharge() != 0)
                part.UseStatistics(false);
        }
    }


    modelgce.SetTemperature(Tch);
    modelgce.SetBaryonChemicalPotential(0.);
    modelgce.SetElectricChemicalPotential(0.);
    modelgce.SetStrangenessChemicalPotential(0.);

    modelgce.FillChemicalPotentials();

    modelgce.CalculateDensities();

    for (int i = 0; i < namesCE.size(); ++i) {
        yieldsGCE.push_back(modelgce.GetDensity(pdgsCE[i], 1) );
    }
}

Prepare the output (on the screen and to a file)

In [8]:
vector<std::ostream*> outs;
outs.push_back(&std::cout);

outs.push_back(NULL);


Prepare the CSM

In [9]:
ThermalModelCanonical modelce(&parts);

if (useWidth)
    modelce.SetUseWidth(ThermalParticle::eBW);
else
    modelce.SetUseWidth(ThermalParticle::ZeroWidth);

modelce.SetStatistics(useQStats);
// Include quantum statistics for mesons only
if (useQStats){
    for (int i = 0; i < modelce.TPS()->Particles().size(); ++i) {
        ThermalParticle &part = modelce.TPS()->Particle(i);
        if (part.BaryonCharge() != 0)
            part.UseStatistics(false);
    }
}

modelce.SetTemperature(Tch);
modelce.SetBaryonChemicalPotential(0.);
modelce.SetElectricChemicalPotential(0.);
modelce.SetStrangenessChemicalPotential(0.);

modelce.SetBaryonCharge(0);
modelce.SetElectricCharge(0);
modelce.SetStrangeness(0);

modelce.FillChemicalPotentials();

Function performing the (logarithmic scale) scan over Vc

In [10]:
void PerformVcScan(ThermalModelBase *model, 
                   const string& filename)
{
    ofstream fout(filename);
    outs[1] = &fout;
    
    for(std::ostream* out : outs) {
        *out << setw(15) << "Vc[fm3]";

        for(auto&& name: namesCE) {
            *out << setw(15) << "(" + name + ")" + "toHML";
        }

        *out << endl;
    }
    
    double Vmin = 4.;
    double Vmax = 30000.;
    int iters = 30;
    double logVmin = log10(Vmin), logVmax = log10(Vmax);
    double dlogV = (logVmax - logVmin) / (iters - 1);
    
    for (double logV = logVmin; logV <= logVmax + 1.e-3; logV += dlogV) {
        double V = pow(10., logV);

        model->SetVolume(V);
        model->SetCanonicalVolume(V);
        model->CalculateDensities();

        for(std::ostream* out : outs) {
            *out << setw(15) << V;

            for (int i = 0; i < namesCE.size(); ++i) {
                *out << setw(15) << model->GetDensity(pdgsCE[i], 1) / yieldsGCE[i];
            }

            *out << endl;
        }
    }
    
    for(std::ostream* out : outs) {
        *out << setw(15) << "HML";

        for (int i = 0; i < namesCE.size(); ++i) {
            *out << setw(15) << 1.;
        }

        *out << endl;
    }
    
    fout.close();
}

BSQ-canonical scan

In [11]:
{
    modelce.ConserveBaryonCharge(true);
    modelce.ConserveElectricCharge(true);
    modelce.ConserveStrangeness(true);
    modelce.CalculateQuantumNumbersRange();
    
    PerformVcScan(&modelce, "out/1906.03145.2_YieldsCSMvsVc.BSQ.dat");
}

        Vc[fm3]     (pi0)toHML     (pi+)toHML      (K+)toHML     (K0S)toHML     (eta)toHML  (KStar+)toHML     (phi)toHML       (p)toHML       (n)toHML  (Lambda)toHML  (Sigma+)toHML  (Sigma+)toHML  (Sigma0)toHML    (Ksi0)toHML    (Ksi-)toHML   (Omega)toHML
              4       0.669018       0.413237       0.174951       0.159071       0.800863       0.114276       0.998931      0.0255632      0.0255083      0.0185856      0.0146149      0.0146922      0.0199692     0.00612922     0.00609696      0.0012018
        5.44105       0.693543       0.460986        0.21286       0.197692       0.820042       0.155878       0.998981      0.0390011      0.0389204       0.029387      0.0238934      0.0240451      0.0313466      0.0103736      0.0103295     0.00223834
        7.40126       0.725643       0.522914       0.268397       0.254326        0.84445       0.216808       0.999054      0.0609029      0.0607837       0.047929      0.0403768      0.0406691      0.0507111      0.0184258      0

BS-canonical scan

In [12]:
{
    modelce.ConserveBaryonCharge(true);
    modelce.ConserveElectricCharge(false);
    modelce.ConserveStrangeness(true);
    modelce.CalculateQuantumNumbersRange();
    
    PerformVcScan(&modelce, "out/1906.03145.2_YieldsCSMvsVc.BS.dat");
}

        Vc[fm3]     (pi0)toHML     (pi+)toHML      (K+)toHML     (K0S)toHML     (eta)toHML  (KStar+)toHML     (phi)toHML       (p)toHML       (n)toHML  (Lambda)toHML  (Sigma+)toHML  (Sigma+)toHML  (Sigma0)toHML    (Ksi0)toHML    (Ksi-)toHML   (Omega)toHML
              4       0.864517       0.851522       0.230946       0.216729       0.977855       0.172572       0.999001      0.0551034      0.0550973      0.0341331      0.0343286      0.0347642      0.0343614      0.0108214      0.0108046     0.00174821
        5.44105        0.87068       0.858363       0.279372       0.266179       0.978588       0.226088       0.999066      0.0769206       0.076913      0.0505143      0.0508327      0.0514527      0.0508794      0.0173682       0.017342     0.00326007
        7.40126       0.878953       0.867539       0.343131        0.33128       0.979598       0.296438       0.999151       0.108044       0.108035      0.0758949      0.0764114      0.0772943       0.076478      0.0287599       

B-canonical scan

In [13]:
{
    modelce.ConserveBaryonCharge(true);
    modelce.ConserveElectricCharge(false);
    modelce.ConserveStrangeness(false);
    modelce.CalculateQuantumNumbersRange();
    
    PerformVcScan(&modelce, "out/1906.03145.2_YieldsCSMvsVc.B.dat");
}

        Vc[fm3]     (pi0)toHML     (pi+)toHML      (K+)toHML     (K0S)toHML     (eta)toHML  (KStar+)toHML     (phi)toHML       (p)toHML       (n)toHML  (Lambda)toHML  (Sigma+)toHML  (Sigma+)toHML  (Sigma0)toHML    (Ksi0)toHML    (Ksi-)toHML   (Omega)toHML
              4       0.942056       0.938539       0.980462        0.98194        0.98417       0.999033              1      0.0843313      0.0843313      0.0843313      0.0843313      0.0843313      0.0843313      0.0843313      0.0843313      0.0843313
        5.44105       0.943957       0.940555       0.981103       0.982533       0.984689       0.999065              1       0.114367       0.114367       0.114367       0.114367       0.114367       0.114367       0.114367       0.114367       0.114367
        7.40126        0.94651       0.943263       0.981964       0.983328       0.985386       0.999108              1       0.154708       0.154708       0.154708       0.154708       0.154708       0.154708       0.154708       

S-canonical scan

In [14]:
{
    modelce.ConserveBaryonCharge(false);
    modelce.ConserveElectricCharge(false);
    modelce.ConserveStrangeness(true);
    modelce.CalculateQuantumNumbersRange();
    
    PerformVcScan(&modelce, "out/1906.03145.2_YieldsCSMvsVc.S.dat");
}

        Vc[fm3]     (pi0)toHML     (pi+)toHML      (K+)toHML     (K0S)toHML     (eta)toHML  (KStar+)toHML     (phi)toHML       (p)toHML       (n)toHML  (Lambda)toHML  (Sigma+)toHML  (Sigma+)toHML  (Sigma0)toHML    (Ksi0)toHML    (Ksi-)toHML   (Omega)toHML
              4       0.913058       0.902891        0.25562       0.241947       0.991187       0.196749        0.99903       0.962605       0.962369       0.181964       0.175741       0.178572       0.175954       0.023028       0.022993     0.00261726
        5.44105       0.919499        0.91008       0.311291       0.298777       0.991855       0.257591       0.999104        0.96544       0.965221       0.241287       0.236564       0.240232       0.236841      0.0387296      0.0386706     0.00519872
        7.40126       0.927834       0.919383       0.383268       0.372239       0.992717       0.336172       0.999199       0.969101       0.968906       0.318192       0.315296       0.319929       0.315645      0.0653362      0

Q-canonical scan

In [15]:
{
    modelce.ConserveBaryonCharge(false);
    modelce.ConserveElectricCharge(true);
    modelce.ConserveStrangeness(false);
    modelce.CalculateQuantumNumbersRange();
    
    PerformVcScan(&modelce, "out/1906.03145.2_YieldsCSMvsVc.Q.dat");
}

        Vc[fm3]     (pi0)toHML     (pi+)toHML      (K+)toHML     (K0S)toHML     (eta)toHML  (KStar+)toHML     (phi)toHML       (p)toHML       (n)toHML  (Lambda)toHML  (Sigma+)toHML  (Sigma+)toHML  (Sigma0)toHML    (Ksi0)toHML    (Ksi-)toHML   (Omega)toHML
              4       0.784591       0.556388       0.537354       0.849141       0.839163       0.436047       0.999608       0.403219       0.768204       0.796371       0.500183       0.488459       0.890289       0.820341       0.519422       0.336416
        5.44105       0.817235       0.619784       0.607746       0.872283         0.8638       0.522542       0.999668       0.477707       0.803156       0.827631       0.576911       0.566987       0.907131       0.847921       0.593197       0.438285
        7.40126       0.854305       0.692412       0.687625       0.898507       0.891725       0.620562       0.999737       0.565464       0.842881        0.86305        0.66385       0.655965       0.926214       0.879171       